# QuickStart
Based on [Pytorch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

- This section runs through the API for common tasks in ML.

## Working with data

PyTorch has 2 primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.

`Dataset` stores the samples and their corresponding labels, and `Dataloader` wraps an iterable around the `Dataset`.

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch has TorchText, TorchVision and TorchAudio, all contain datasets.

We will use a TorchVision dataset.

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO etc

Here is the [full list](https://pytorch.org/vision/stable/datasets.html)

We'll use a 3 datasets to get the gist: FashionMNIST.

Every TorchVision Dataset includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [ ]:
# download training data from open datasets

train_data = 